**DATA**
<br>
src: https://snap.stanford.edu/data/twitter_combined.txt.gz
<br>
content: Directed edges

Download data

In [1]:
import requests

url = 'https://snap.stanford.edu/data/twitter_combined.txt.gz'
file_name = 'twitter_combined.txt.gz'

with requests.get(url, stream=True) as r:
    with open(file_name, 'wb') as f:
        r.raise_for_status()
        for chunk in r.iter_content():
            f.write(chunk)

In [2]:
!rm -rf twitter_combined.txt

In [3]:
!gzip -d twitter_combined.txt.gz

**Pyspark - computing page ranks**

Spark entry-point

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession. \
        builder. \
        appName("PythonPageRank"). \
        getOrCreate()

sc = spark.sparkContext

Import method to compute page ranks

In [6]:
from page_rank import calculate_page_ranks

Compute page ranks

In [7]:
%%time
page_ranks = calculate_page_ranks(sc, 'twitter_combined.txt', 1, verbose=True)

Completed iteration: 0 Norm of change: 0.009623599223975543
Completed iteration: 1 Norm of change: 0.003958334615176089
Completed iteration: 2 Norm of change: 0.002585182373956529
Completed iteration: 3 Norm of change: 0.002109001588015743
Completed iteration: 4 Norm of change: 0.0017404471812438613
Completed iteration: 5 Norm of change: 0.0014765669504370998
Completed iteration: 6 Norm of change: 0.0012205933413020449
Completed iteration: 7 Norm of change: 0.0010400717007814906
Completed iteration: 8 Norm of change: 0.0008599520626447965
Completed iteration: 9 Norm of change: 0.0007338868885612816
Completed iteration: 10 Norm of change: 0.0006073244483828667
Completed iteration: 11 Norm of change: 0.0005187391517934819
Completed iteration: 12 Norm of change: 0.00042981877173917105
Completed iteration: 13 Norm of change: 0.0003673468484928437
Completed iteration: 14 Norm of change: 0.00030481530742298117
Completed iteration: 15 Norm of change: 0.00026062913338956124
Completed iteration

Sort pages by page ranks (descending order)

In [8]:
%%time
page_ranks_desc = page_ranks.sortBy(lambda page: page[1], ascending=False)

CPU times: user 16.1 ms, sys: 7.99 ms, total: 24.1 ms
Wall time: 939 ms


Top 5 pages

In [9]:
%%time
print(*page_ranks_desc.take(5), sep='\n')

('115485051', 0.009848538545711827)
('116485573', 0.008401366206669711)
('813286', 0.002014479264569119)
('40981798', 0.0019633251619017664)
('11348282', 0.0018254172503175535)
CPU times: user 12.3 ms, sys: 1.03 ms, total: 13.3 ms
Wall time: 787 ms


Save page ranks

In [10]:
!rm -rf twitter_page_ranks

In [11]:
%%time
page_ranks_desc.map(lambda x: x[0] + ' ' + str(x[1])). \
                saveAsTextFile('twitter_page_ranks')

CPU times: user 14.3 ms, sys: 1.06 ms, total: 15.3 ms
Wall time: 1.44 s
